In [476]:
from pathlib import Path
import pandas as pd

In [477]:
def dataframe_to_markdown(df):
    headers = list(df.columns)
    markdown = "| " + " | ".join(headers) + " |\n"
    markdown += "| " + " | ".join(["---"] * len(headers)) + " |\n"

    for _, row in df.iterrows():
        markdown += "| " + " | ".join(str(cell) for cell in row) + " |\n"
    return markdown

In [ ]:
dataset = "HelocDataset"
method = "PUMAL/E19"
# model_name = "MultilayerPerceptron"
model_name = "MultinomialLogisticRegression"

root = f"../models/{dataset}/{method}"

In [479]:
metrics = []
time_files = []

In [480]:
for i in range(5):
    path = Path(root) / f"fold_{i}" / f"cf_metrics_{model_name}.csv"
    df = pd.read_csv(path)
    metrics.append(df)

In [481]:
merged_df = pd.concat(metrics, axis=0, ignore_index=True)

In [482]:
mean_ = merged_df.mean(axis=0)
std_ = merged_df.std(axis=0)

In [483]:
def format_mean_std(m, s):
    return f"{m:.2f} ± {s:.2f}"


# Create the merged table
formatted = pd.DataFrame(
    {col: [format_mean_std(mean_[col], std_[col])] for col in merged_df.columns}
).T

In [484]:
used_metrics = [
    "K_vectors",
    "coverage",
    "validity",
    "proximity_continuous_euclidean",
    "isolation_forest_scores_cf",
    "log_density_cf",
    # "lof_scores_cf",
    "prob_plausibility",
    "pairwise_cosine_sim_min",
]

In [485]:
transposed = formatted.T
transposed = transposed[used_metrics]

In [486]:
transposed

,K_vectors,coverage,validity,proximity_continuous_euclidean,isolation_forest_scores_cf,log_density_cf,prob_plausibility,pairwise_cosine_sim_min
0,290.00 ± 31.80,1.00 ± 0.00,1.00 ± 0.00,0.44 ± 0.06,0.08 ± 0.00,33.32 ± 0.41,0.99 ± 0.01,0.09 ± 0.02


In [487]:
print(dataframe_to_markdown(transposed))

| K_vectors | coverage | validity | proximity_continuous_euclidean | isolation_forest_scores_cf | log_density_cf | prob_plausibility | pairwise_cosine_sim_min |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 290.00 ± 31.80 | 1.00 ± 0.00 | 1.00 ± 0.00 | 0.44 ± 0.06 | 0.08 ± 0.00 | 33.32 ± 0.41 | 0.99 ± 0.01 | 0.09 ± 0.02 |

